In [1]:
import requests
import time 
import pandas as pd
import bs4
from bs4 import BeautifulSoup

from time import sleep
from tqdm import tqdm

In [2]:
locations=['cairo','Banha','Maadi','Nasr+City','Giza','Toukh','Qalyub']
postions =['Python+developer','data+scientist','data+science','data+analyst','data+analysis','data+entry','data+engineer','database']
pages = list(range(0,110,10))

In [3]:
def extract(loc, pos, page):

  URL='http://eg.indeed.com/jobs?q='+str(pos)+'&l=' + str(loc)+'&start='+str(page)+'&sort=date'
  response = requests.get(URL)
  soup = BeautifulSoup(response.text,'lxml')
  return soup, URL

In [4]:

def company_posts(soup):
  jobs = []
  for i in soup.find_all('a',{'class':"tapItem"}):

    for spans in i.select('h2.jobTitle span'):
      try:
        job_title = spans['title']
      except:
        pass
    
    location = i.find('div', {'class': 'companyLocation'}).text.strip()
    company_name = i.find(('a', {'data-tn-element': 'companyName'})).text.strip()    
    post_date = i.find('span',{'class':'date'}).text.strip().split('d')[-1]
    summary = i.find('div',{'class':'job-snippet'}).text.strip()
    
    p = i.find(href=True)
    v = p['href']
    f=str(v).replace('&amp','&')
    link= str('http://eg.indeed.com' + str(f)) 

    job = {'job_title':job_title,
            'location':location,
            'compny_name':company_name, 
            'post_date':post_date,
            'summary':summary,
            'Qualification_link':link
            }
    jobs.append(job)
    
  return jobs

In [5]:
start_time = time.time()
df_list = []
for page in tqdm(pages):
  for loc in locations:
    for pos in postions:
      mysoup,URL = extract(loc,pos, page)
      #print(mysoup)
      jobs= company_posts(mysoup)
      df = pd.DataFrame(jobs)
      #print(df)
      df_list.append(df)
      #print(df_list)
print( '\nThe process taks {:.2f} sec'.format( (time.time() - start_time)) )

100%|██████████| 11/11 [04:00<00:00, 21.91s/it]


The process taks 241.00 sec


In [7]:
mydf=pd.concat(df_list,ignore_index=True )

In [10]:
mydf.to_csv('indeed-jobs-egypt')

In [11]:
df = pd.read_csv('indeed-jobs-egypt')
df

,Unnamed: 0,job_title,location,compny_name,post_date,summary,Qualification_link
0,0,Linux Developer Experience Lead,القاهرة,وظائف Canonical - Jobs,اليوم,"In this new initiative, we want to raise the b...",http://eg.indeed.com/%D9%88%D8%B8%D8%A7%D8%A6%...
1,1,"VP, Software Engineering",القاهرة,وظائف Canonical - Jobs,اليوم,"Our online services, including our systems man...",http://eg.indeed.com/%D9%88%D8%B8%D8%A7%D8%A6%...
2,2,Senior Software Developer (MEAN/MERN),القاهرة,Schneider Electric,منذ يوم 1,"In this role, you will be responsible for over...",http://eg.indeed.com/cmp/Schneider-Electric
3,3,Senior C++ SW Developer,القاهرة,وظائف Luxoft,منذ 13 يوم,The project life-cycle is covering all the sta...,http://eg.indeed.com/%D9%88%D8%B8%D8%A7%D8%A6%...
4,4,"Software Engineer, Cloud Integrations, Commerc...",القاهرة,وظائف Canonical - Jobs,منذ 12 يوم,5+ years of experience as a software developer...,http://eg.indeed.com/%D9%88%D8%B8%D8%A7%D8%A6%...
...,...,...,...,...,...,...,...
2281,2281,Senior Data Scientist,القاهرة,Careem,منذ 198 يوم,Experience mentoring other data scientists.\n5...,http://eg.indeed.com/cmp/Careem
2282,2282,Data Scientist,القاهرة,وظائف CyShield,منذ 243 يوم,Collating and cleaning data from various entit...,http://eg.indeed.com/%D9%88%D8%B8%D8%A7%D8%A6%...
2283,2283,Performance Management Analyst,القاهرة,وظائف Talabat,منذ 246 يوم,Take full ownership and create data-driven ins...,http://eg.indeed.com/%D9%88%D8%B8%D8%A7%D8%A6%...
2284,2284,Sr. Analyst Performance Management,القاهرة,وظائف Talabat,منذ 246 يوم,Take full ownership and create data-driven ins...,http://eg.indeed.com/%D9%88%D8%B8%D8%A7%D8%A6%...
